In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import gensim as gs
import tensorflow as tf
import warnings
import tensorflow_hub as hub
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
warnings.filterwarnings('ignore')
from sklearn.metrics import pairwise_distances
from heapq import nsmallest

In [2]:
test_articles = pd.read_csv('../dataset/test_articles_ourdata.csv', delimiter=',')

In [3]:
fixed_index = pd.read_csv('../dataset/fixed_test_set_ourdata.csv', delimiter=',')

In [4]:
embedding_model = Doc2Vec.load('../pre_trained_models/doc2vec.bin')

In [5]:
def get_accuracy(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    
    for table_ID in ranked_tables_ID:
        
        if table_ID[0] == ID_goal:
    
            accuracy = 1
            break;

    return accuracy

In [6]:
def get_mrr(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    index_match = 1

    for idTable in ranked_tables_ID:
        
        if idTable[0] == ID_goal:
    
            accuracy = 1/index_match
            break;
        
        index_match = index_match + 1
   
    return accuracy

In [7]:
test_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948 entries, 0 to 947
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   article_id                948 non-null    int64 
 1   article_title             948 non-null    object
 2   article_full_text         948 non-null    object
 3   article_meta_description  948 non-null    object
 4   article_summary           948 non-null    object
 5   article_keywords          948 non-null    object
 6   article_meta_key_words    662 non-null    object
 7   article_tags              207 non-null    object
dtypes: int64(1), object(7)
memory usage: 59.4+ KB


In [8]:
fixed_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94721 entries, 0 to 94720
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   label_index              94721 non-null  int64 
 1   table_page_id            94721 non-null  int64 
 2   table_page_title         94721 non-null  object
 3   table_page_main_passage  94721 non-null  object
 4   table_page_keywords      94721 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.6+ MB


In [9]:
evaluate_topk = [1,5,10,20,50]

result = []

#evaluating each topk value
for TOP_K in evaluate_topk:

    accuracy = []
    mrr = []

    for i, row in tqdm(test_articles.iterrows()):

        #current article values
        article_ID = row['article_id']
        article_title_text = str(row['article_title'])
        article_meta_description_text = str(row['article_meta_description'])
        article_keywords_text = str(row['article_keywords'])

        #embedding and model variables
        article_title1 = []
        article_title2 = []
        article_title3 = []
        title_table1 = []
        title_table2 = []
        title_table3 = []
        ranked_tables_model = []

        #return index
        return_index = fixed_index.loc[fixed_index['label_index'] == row['article_id']]
        
        vector_words = tknzr.tokenize(article_title_text)
        word_embedding = embedding_model.infer_vector(vector_words)
        article_title1.append(word_embedding)
        
        vector_words = tknzr.tokenize(article_meta_description_text)
        word_embedding = embedding_model.infer_vector(vector_words)
        article_title2.append(word_embedding)
        
        vector_words = tknzr.tokenize(article_keywords_text)
        word_embedding = embedding_model.infer_vector(vector_words)
        article_title3.append(word_embedding)

        #creating embedding 
        for i, row in return_index.iterrows():
            
            vector_words = tknzr.tokenize(str(row['table_page_title']))
            word_embedding = embedding_model.infer_vector(vector_words)
            title_table1.append(word_embedding)
            
            vector_words = tknzr.tokenize(str(row['table_page_main_passage']))
            word_embedding = embedding_model.infer_vector(vector_words)
            title_table2.append(word_embedding)
            
            vector_words = tknzr.tokenize(str(row['table_page_keywords']))
            word_embedding = embedding_model.infer_vector(vector_words)
            title_table3.append(word_embedding)
        
        distance_vector1 = pairwise_distances(article_title1, title_table1, metric='cosine')
        distance_vector2 = pairwise_distances(article_title2, title_table2, metric='cosine')
        distance_vector3 = pairwise_distances(article_title3, title_table3, metric='cosine')
        
        final_vector = (np.array(distance_vector1) + np.array(distance_vector2) + np.array(distance_vector3)) / 3.0
        
        #creating the final dataframe
        for i in range(0,len(final_vector[0])):

            ranked_tables_model.append([return_index.iloc[i]['table_page_id'],return_index.iloc[i]['table_page_title'],final_vector[0][i]]) 

        data_frame = pd.DataFrame(ranked_tables_model, columns = ['table_ID', 'table_title','table_ranking']) 
        data_frame_sorting = data_frame.sort_values('table_ranking')
        
        selected_top = data_frame_sorting.head(TOP_K)
#         max_score = selected_top['table_ranking'].max()
#         draw_tables_socres = data_frame_sorting[data_frame_sorting['table_ranking'] <= max_score]
        final_ranked_tables = selected_top.iloc[:,0:1].values

        #getting topk accuracy
        accuracy.append(get_accuracy(article_ID, final_ranked_tables))

        #testing mean reciprocal rank at k = 50
        if TOP_K == 50:

            mrr.append(get_mrr(article_ID, final_ranked_tables))

    result.append(["Acc@"+str(TOP_K),str(round(np.mean(accuracy),4))])

print("")
print(result[0])
print(result[1])
print(result[2])
print(result[3])
print(result[4])
print("MRR: "+str(round(np.mean(mrr),4)) )
print(mrr)
print("")

948it [09:09,  1.73it/s]
948it [09:07,  1.73it/s]
948it [09:02,  1.75it/s]
948it [08:05,  1.95it/s]
948it [07:22,  2.14it/s]


['Acc@1', '0.4589']
['Acc@5', '0.7342']
['Acc@10', '0.8291']
['Acc@20', '0.9103']
['Acc@50', '0.9747']
MRR: 0.5946
[0.1, 0.037037037037037035, 0.5, 0.05263157894736842, 0.5, 1.0, 0.5, 0.5, 1.0, 0.14285714285714285, 0.1, 0.3333333333333333, 0.037037037037037035, 1.0, 0.16666666666666666, 0.08333333333333333, 0.25, 0.2, 0.3333333333333333, 1.0, 1.0, 0.0625, 1.0, 0.03571428571428571, 0.3333333333333333, 0.041666666666666664, 0.5, 0.2, 0.058823529411764705, 1.0, 0.3333333333333333, 1.0, 0.2, 1.0, 0.5, 0.25, 0.041666666666666664, 1.0, 1.0, 1.0, 1.0, 0.2, 0.047619047619047616, 0.1, 1.0, 0.2, 0.1, 0.16666666666666666, 0.14285714285714285, 0.25, 0, 1.0, 0.2, 0.14285714285714285, 0.16666666666666666, 0.5, 1.0, 1.0, 1.0, 0.3333333333333333, 0.3333333333333333, 0.5, 0.02040816326530612, 1.0, 0.1, 0.09090909090909091, 1.0, 1.0, 1.0, 1.0, 0.09090909090909091, 1.0, 0.041666666666666664, 0.05555555555555555, 1.0, 0.5, 1.0, 1.0, 0.25, 0.06666666666666667, 1.0, 1.0, 0.5, 0.03333333333333333, 0.125, 1.